# TraffixVision

## 1. Counting the Vehicles

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Open video file
video_path = "d1.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define counting box (adjust these coordinates as needed)
count_box = {
    'x1': int(width * 0.2),
    'y1': int(height * 0.5),
    'x2': int(width * 0.8),
    'y2': int(height * 0.8)
}

# Initialize vehicle count
vehicle_count = 0

# Define vehicle classes
vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# Create video writer object
output_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Create directory for saved frames
os.makedirs("saved_frames", exist_ok=True)

frame_count = 0
tracked_objects = {}

def box_center(box):
    return ((box[0] + box[2]) // 2, (box[1] + box[3]) // 2)

def point_in_box(point, box):
    return box['x1'] < point[0] < box['x2'] and box['y1'] < point[1] < box['y2']

def iou(box1, box2):
    # Calculate intersection over union
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection)
    return iou

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Run YOLOv8 inference
    results = model(frame)

    # Draw counting box
    cv2.rectangle(frame, (count_box['x1'], count_box['y1']), (count_box['x2'], count_box['y2']), (255, 0, 0), 2)

    current_objects = {}

    # Process detections
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            if cls in vehicle_classes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                current_box = (x1, y1, x2, y2)
                center = box_center(current_box)
                
                matched = False
                for obj_id, obj in tracked_objects.items():
                    if iou(current_box, obj['box']) > 0.5:  # If IOU > 0.5, consider it the same object
                        current_objects[obj_id] = {'box': current_box, 'center': center, 'counted': obj['counted']}
                        matched = True
                        break
                
                if not matched:
                    new_id = max(tracked_objects.keys()) + 1 if tracked_objects else 0
                    current_objects[new_id] = {'box': current_box, 'center': center, 'counted': False}
    
    # Count vehicles
    for obj_id, obj in current_objects.items():
        if not obj['counted'] and point_in_box(obj['center'], count_box):
            vehicle_count += 1
            obj['counted'] = True
    
    tracked_objects = current_objects

    # Display vehicle count
    cv2.putText(frame, f"Vehicle Count: {vehicle_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write frame to output video
    out.write(frame)

    # Save frame as image every 30 frames
    if frame_count % 30 == 0:
        cv2.imwrite(f"saved_frames/frame_{frame_count}.jpg", frame)

    print(f"Processed frame {frame_count}, Current vehicle count: {vehicle_count}", end='\r')

# Release resources
cap.release()
out.release()

print(f"\nProcessing complete. Total vehicle count: {vehicle_count}")

100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:02<00:00, 2.27MB/s]



0: 384x640 (no detections), 151.8ms
Speed: 9.6ms preprocess, 151.8ms inference, 9.6ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 1, Current vehicle count: 0
0: 384x640 (no detections), 60.3ms
Speed: 3.0ms preprocess, 60.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 2, Current vehicle count: 0
0: 384x640 1 car, 63.1ms
Speed: 3.6ms preprocess, 63.1ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 3, Current vehicle count: 0
0: 384x640 1 car, 76.6ms
Speed: 2.0ms preprocess, 76.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 4, Current vehicle count: 0
0: 384x640 2 cars, 66.8ms
Speed: 2.1ms preprocess, 66.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 5, Current vehicle count: 0
0: 384x640 2 cars, 66.9ms
Speed: 2.0ms preprocess, 66.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 6, Curr

## 2. Modifying to add Prediction arrows through lane detection

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # You can use 'yolov8s.pt' for a smaller model if needed

# Function to detect vehicles
def detect_vehicles(frame):
    results = model(frame)
    boxes = []
    for result in results:
        for box in result.boxes:
            cls = box.cls.item()
            conf = box.conf.item()
            if conf > 0.5 and cls in [2, 3, 5, 7]:  # Class IDs for car, motorcycle, bus, truck
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                boxes.append([x1, y1, x2 - x1, y2 - y1])
    return boxes

# Load the video file
video_path = 'd1.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using 'mp4v' codec for MP4 format
out = cv2.VideoWriter('output_video1.mp4', fourcc, fps, (frame_width, frame_height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Reached the end of the video or failed to read frame.")
        break

    frame_count += 1
    print(f"Processing frame {frame_count}")

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detector
    edges = cv2.Canny(blurred, 50, 150)

    # Use Hough Transform to detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, minLineLength=100, maxLineGap=50)

    # Create a copy of the original frame to draw on
    line_image = np.zeros_like(frame)

    # Detect vehicles
    vehicles = detect_vehicles(frame)

    # Create mask to exclude vehicles from line drawing
    vehicle_mask = np.zeros_like(edges)
    for (x, y, w, h) in vehicles:
        cv2.rectangle(vehicle_mask, (x, y), (x + w, y + h), 255, thickness=cv2.FILLED)

    # Draw detected lines in blue, excluding vehicle areas
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if not (vehicle_mask[y1, x1] or vehicle_mask[y2, x2]):
                    cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 5)

    # Combine the line image with the original frame without blending
    mask = np.where((line_image != 0))
    combo = frame.copy()
    combo[mask] = line_image[mask]

    # Predict direction based on the lanes and draw arrows on vehicles
    height, width = combo.shape[:2]

    if lines is not None:
        left_lane = []
        right_lane = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x1 < width // 2 and x2 < width // 2:
                    left_lane.append((x1, y1, x2, y2))
                elif x1 > width // 2 and x2 > width // 2:
                    right_lane.append((x1, y1, x2, y2))

    for (x, y, w, h) in vehicles:
        center_x = x + w // 2
        if left_lane and center_x < width // 3:
            cv2.arrowedLine(combo, (center_x, y + h), (center_x - 50, y + h + 50), (0, 0, 255), 5)
        elif right_lane and center_x > 2 * width // 3:
            cv2.arrowedLine(combo, (center_x, y + h), (center_x + 50, y + h + 50), (0, 0, 255), 5)
        else:
            cv2.arrowedLine(combo, (center_x, y + h), (center_x, y + h + 50), (0, 0, 255), 5)

    # Write the processed frame to the output video
    out.write(combo)

print("Finished processing all frames.")

# Release everything if job is finished
cap.release()
out.release()


Processing frame 1

0: 384x640 (no detections), 65.6ms
Speed: 3.0ms preprocess, 65.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2

0: 384x640 (no detections), 72.5ms
Speed: 2.0ms preprocess, 72.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 3

0: 384x640 1 car, 61.6ms
Speed: 2.0ms preprocess, 61.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 4

0: 384x640 1 car, 59.7ms
Speed: 2.0ms preprocess, 59.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 5

0: 384x640 2 cars, 63.2ms
Speed: 3.0ms preprocess, 63.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 6

0: 384x640 2 cars, 73.6ms
Speed: 1.0ms preprocess, 73.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 7

0: 384x640 1 car, 84.2ms
Speed: 2.9ms preprocess, 84.2ms inference, 4.0ms postprocess per image at shape (

## 3. Reducing the blue lines that affect the video visibality

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # You can use 'yolov8s.pt' for a smaller model if needed

# Function to detect vehicles
def detect_vehicles(frame):
    results = model(frame)
    boxes = []
    for result in results:
        for box in result.boxes:
            cls = box.cls.item()
            conf = box.conf.item()
            if conf > 0.5 and cls in [2, 3, 5, 7]:  # Class IDs for car, motorcycle, bus, truck
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                boxes.append([x1, y1, x2 - x1, y2 - y1])
    return boxes

# Function to apply a perspective transform to get a bird's-eye view
def perspective_transform(frame):
    height, width = frame.shape[:2]
    src = np.float32([
        [width * 0.45, height * 0.65],
        [width * 0.55, height * 0.65],
        [width * 0.1, height],
        [width * 0.9, height]
    ])
    dst = np.float32([
        [width * 0.2, 0],
        [width * 0.8, 0],
        [width * 0.2, height],
        [width * 0.8, height]
    ])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(frame, M, (width, height))
    return warped, M

# Function to draw lane lines on the frame
def draw_lane_lines(frame, lines, Minv):
    if lines is None:
        return frame
    for line in lines:
        for x1, y1, x2, y2 in line:
            pts = np.array([[[x1, y1], [x2, y2]]], dtype=np.float32)
            pts = cv2.perspectiveTransform(pts, Minv)
            x1, y1 = map(int, pts[0][0])
            x2, y2 = map(int, pts[0][1])
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 5)
    return frame

# Load the video file
video_path = 'd1.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using 'mp4v' codec for MP4 format
out = cv2.VideoWriter('output_video2.mp4', fourcc, fps, (frame_width, frame_height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Reached the end of the video or failed to read frame.")
        break

    frame_count += 1
    print(f"Processing frame {frame_count}")

    # Apply perspective transform to get bird's-eye view
    warped, M = perspective_transform(frame)

    # Convert warped frame to grayscale
    gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detector
    edges = cv2.Canny(blurred, 50, 150)

    # Use morphological operations to clean up the edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Use Hough Transform to detect lines with adjusted parameters
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, minLineLength=100, maxLineGap=50)

    # Create a mask to exclude vehicles from line drawing
    vehicles = detect_vehicles(frame)
    vehicle_mask = np.zeros_like(edges)
    for (x, y, w, h) in vehicles:
        cv2.rectangle(vehicle_mask, (x, y), (x + w, y + h), 255, thickness=cv2.FILLED)

    # Transform back to original perspective
    Minv = cv2.getPerspectiveTransform(np.float32([
        [frame_width * 0.2, 0],
        [frame_width * 0.8, 0],
        [frame_width * 0.2, frame_height],
        [frame_width * 0.8, frame_height]
    ]), np.float32([
        [frame_width * 0.45, frame_height * 0.65],
        [frame_width * 0.55, frame_height * 0.65],
        [frame_width * 0.1, frame_height],
        [frame_width * 0.9, frame_height]
    ]))

    # Draw detected lines in blue, excluding vehicle areas
    line_image = np.copy(frame)
    line_image = draw_lane_lines(line_image, lines, Minv)

    # Combine the line image with the original frame
    combo = cv2.addWeighted(frame, 0.8, line_image, 1, 0)

    # Predict direction based on the lanes and draw arrows on vehicles
    height, width = combo.shape[:2]

    left_lane = []
    right_lane = []
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x1 < width // 2 and x2 < width // 2:
                    left_lane.append((x1, y1, x2, y2))
                elif x1 > width // 2 and x2 > width // 2:
                    right_lane.append((x1, y1, x2, y2))

    for (x, y, w, h) in vehicles:
        center_x = x + w // 2
        if left_lane and center_x < width // 3:
            cv2.arrowedLine(combo, (center_x, y + h), (center_x - 50, y + h + 50), (0, 0, 255), 5)
        elif right_lane and center_x > 2 * width // 3:
            cv2.arrowedLine(combo, (center_x, y + h), (center_x + 50, y + h + 50), (0, 0, 255), 5)
        else:
            cv2.arrowedLine(combo, (center_x, y + h), (center_x, y + h + 50), (0, 0, 255), 5)

    # Write the processed frame to the output video
    out.write(combo)

print("Finished processing all frames.")

# Release everything if job is finished
cap.release()
out.release()

Processing frame 1

0: 384x640 (no detections), 63.4ms
Speed: 3.0ms preprocess, 63.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2

0: 384x640 (no detections), 57.2ms
Speed: 2.0ms preprocess, 57.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 3

0: 384x640 1 car, 134.3ms
Speed: 8.0ms preprocess, 134.3ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 4

0: 384x640 1 car, 86.9ms
Speed: 3.9ms preprocess, 86.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 5

0: 384x640 2 cars, 97.6ms
Speed: 3.0ms preprocess, 97.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 6

0: 384x640 2 cars, 117.4ms
Speed: 3.0ms preprocess, 117.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 7

0: 384x640 1 car, 115.1ms
Speed: 4.0ms preprocess, 115.1ms inference, 3.0ms postprocess per image at s

## 4. Integration of Vehivle count and Direction prediction

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Open video file
video_path = "d1.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define counting box (adjust these coordinates as needed)
count_box = {
    'x1': int(width * 0.2),
    'y1': int(height * 0.5),
    'x2': int(width * 0.8),
    'y2': int(height * 0.8)
}

# Initialize vehicle count
vehicle_count = 0

# Define vehicle classes
vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# Create video writer object
output_path = "output_video3.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Create directory for saved frames
os.makedirs("saved_frames", exist_ok=True)

frame_count = 0
tracked_objects = {}

def box_center(box):
    return ((box[0] + box[2]) // 2, (box[1] + box[3]) // 2)

def point_in_box(point, box):
    return box['x1'] < point[0] < box['x2'] and box['y1'] < point[1] < box['y2']

def iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area1 + area2 - intersection)
    return iou

def perspective_transform(frame):
    height, width = frame.shape[:2]
    src = np.float32([
        [width * 0.45, height * 0.65],
        [width * 0.55, height * 0.65],
        [width * 0.1, height],
        [width * 0.9, height]
    ])
    dst = np.float32([
        [width * 0.2, 0],
        [width * 0.8, 0],
        [width * 0.2, height],
        [width * 0.8, height]
    ])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(frame, M, (width, height))
    return warped, M

def draw_lane_lines(frame, lines, Minv):
    if lines is None:
        return frame
    for line in lines:
        for x1, y1, x2, y2 in line:
            pts = np.array([[[x1, y1], [x2, y2]]], dtype=np.float32)
            pts = cv2.perspectiveTransform(pts, Minv)
            x1, y1 = map(int, pts[0][0])
            x2, y2 = map(int, pts[0][1])
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 5)
    return frame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Apply perspective transform to get bird's-eye view
    warped, M = perspective_transform(frame)

    # Convert warped frame to grayscale
    gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detector
    edges = cv2.Canny(blurred, 50, 150)

    # Use morphological operations to clean up the edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Use Hough Transform to detect lines with adjusted parameters
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, minLineLength=100, maxLineGap=50)

    # Transform back to original perspective
    Minv = cv2.getPerspectiveTransform(np.float32([
        [width * 0.2, 0],
        [width * 0.8, 0],
        [width * 0.2, height],
        [width * 0.8, height]
    ]), np.float32([
        [width * 0.45, height * 0.65],
        [width * 0.55, height * 0.65],
        [width * 0.1, height],
        [width * 0.9, height]
    ]))

    # Draw detected lines in blue
    line_image = np.copy(frame)
    line_image = draw_lane_lines(line_image, lines, Minv)

    # Combine the line image with the original frame
    combo = cv2.addWeighted(frame, 0.8, line_image, 1, 0)

    # Run YOLOv8 inference
    results = model(frame)

    # Draw counting box
    cv2.rectangle(combo, (count_box['x1'], count_box['y1']), (count_box['x2'], count_box['y2']), (255, 0, 0), 2)

    current_objects = {}

    left_lane = []
    right_lane = []
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x1 < width // 2 and x2 < width // 2:
                    left_lane.append((x1, y1, x2, y2))
                elif x1 > width // 2 and x2 > width // 2:
                    right_lane.append((x1, y1, x2, y2))

    # Process detections
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            if cls in vehicle_classes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # Draw bounding box
                cv2.rectangle(combo, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                current_box = (x1, y1, x2, y2)
                center = box_center(current_box)
                
                matched = False
                for obj_id, obj in tracked_objects.items():
                    if iou(current_box, obj['box']) > 0.5:  # If IOU > 0.5, consider it the same object
                        current_objects[obj_id] = {'box': current_box, 'center': center, 'counted': obj['counted']}
                        matched = True
                        break
                
                if not matched:
                    new_id = max(tracked_objects.keys()) + 1 if tracked_objects else 0
                    current_objects[new_id] = {'box': current_box, 'center': center, 'counted': False}
                
                # Predict direction and draw arrow
                center_x = (x1 + x2) // 2
                if left_lane and center_x < width // 3:
                    cv2.arrowedLine(combo, (center_x, y2), (center_x - 50, y2 + 50), (0, 0, 255), 5)
                elif right_lane and center_x > 2 * width // 3:
                    cv2.arrowedLine(combo, (center_x, y2), (center_x + 50, y2 + 50), (0, 0, 255), 5)
                else:
                    cv2.arrowedLine(combo, (center_x, y2), (center_x, y2 + 50), (0, 0, 255), 5)
    
    # Count vehicles
    for obj_id, obj in current_objects.items():
        if not obj['counted'] and point_in_box(obj['center'], count_box):
            vehicle_count += 1
            obj['counted'] = True
    
    tracked_objects = current_objects

    # Display vehicle count
    cv2.putText(combo, f"Vehicle Count: {vehicle_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write frame to output video
    out.write(combo)

    # Save frame as image every 30 frames
    if frame_count % 30 == 0:
        cv2.imwrite(f"saved_frames/frame_{frame_count}.jpg", combo)

    print(f"Processed frame {frame_count}, Current vehicle count: {vehicle_count}", end='\r')

# Release resources
cap.release()
out.release()
print(f"\nProcessing complete. Total vehicle count: {vehicle_count}")


0: 384x640 (no detections), 61.7ms
Speed: 3.0ms preprocess, 61.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 1, Current vehicle count: 0
0: 384x640 (no detections), 51.1ms
Speed: 2.0ms preprocess, 51.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 2, Current vehicle count: 0
0: 384x640 1 car, 47.5ms
Speed: 2.0ms preprocess, 47.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 3, Current vehicle count: 0
0: 384x640 1 car, 55.0ms
Speed: 1.9ms preprocess, 55.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 4, Current vehicle count: 0
0: 384x640 2 cars, 56.9ms
Speed: 1.0ms preprocess, 56.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 5, Current vehicle count: 0
0: 384x640 2 cars, 109.8ms
Speed: 3.7ms preprocess, 109.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 6, Curr

## 5. Modifying the direction prediction and calculating the average time of a vehicle between two lines

In [15]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
from collections import defaultdict, deque

def detect_lanes(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply Canny edge detection
    edges = cv2.Canny(blur, 50, 150)
    
    # Define region of interest
    height, width = frame.shape[:2]
    roi_vertices = np.array([[(0, height), (width/2, height/2), (width, height)]], dtype=np.int32)
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, roi_vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)
    
    # Apply Hough Transform
    lines = cv2.HoughLinesP(masked_edges, rho=2, theta=np.pi/180, threshold=100, minLineLength=40, maxLineGap=5)
    
    return lines

def draw_lanes(frame, lines):
    lane_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(lane_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return cv2.addWeighted(frame, 0.8, lane_image, 1, 1)

def determine_direction(center_x, frame_width):
    left_threshold = frame_width / 3
    right_threshold = 2 * frame_width / 3
    if center_x < left_threshold:
        return "left"
    elif center_x > right_threshold:
        return "right"
    else:
        return "straight"

def main():
    # Load YOLOv8 model
    try:
        model = YOLO('yolov8n.pt')
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return

    # Open video file
    video_path = "d1.mp4"
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define two lines for timing
    start_line = {'y': int(height * 0.3)}
    end_line = {'y': int(height * 0.8)}

    # Define counting box
    count_box = {
        'x1': int(width * 0.2),
        'y1': int(height * 0.5),
        'x2': int(width * 0.8),
        'y2': int(height * 0.8)
    }

    # Initialize vehicle tracking and counting
    vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck
    tracked_objects = {}
    vehicle_frames = defaultdict(dict)
    travel_times = deque(maxlen=50)  # Store last 50 travel times
    vehicle_count = 0

    # Create video writer object
    output_path = "output_video_timed_with_lanes.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Create directory for saved frames
    os.makedirs("saved_frames", exist_ok=True)

    frame_count = 0

    def box_center(box):
        return ((box[0] + box[2]) // 2, (box[1] + box[3]) // 2)

    def point_in_box(point, box):
        return box['x1'] < point[0] < box['x2'] and box['y1'] < point[1] < box['y2']

    def iou(box1, box2):
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        return intersection / float(area1 + area2 - intersection)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        # Detect lanes
        lines = detect_lanes(frame)
        
        # Draw lanes on frame
        draw_frame = draw_lanes(frame, lines)

        # Run YOLOv8 inference
        results = model(frame)

        # Draw timing lines (red)
        cv2.line(draw_frame, (0, start_line['y']), (width, start_line['y']), (0, 0, 255), 2)
        cv2.line(draw_frame, (0, end_line['y']), (width, end_line['y']), (0, 0, 255), 2)

        # Draw counting box (blue)
        cv2.rectangle(draw_frame, (count_box['x1'], count_box['y1']), (count_box['x2'], count_box['y2']), (255, 0, 0), 2)

        current_objects = {}

        # Process detections
        for r in results:
            boxes = r.boxes
            for box in boxes:
                cls = int(box.cls[0])
                if cls in vehicle_classes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    
                    # Draw bounding box
                    cv2.rectangle(draw_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    
                    current_box = (x1, y1, x2, y2)
                    center = box_center(current_box)
                    
                    # Determine direction
                    direction = determine_direction(center[0], width)
                    
                    matched = False
                    for obj_id, obj in tracked_objects.items():
                        if iou(current_box, obj['box']) > 0.5:
                            current_objects[obj_id] = {'box': current_box, 'center': center, 'counted': obj['counted'], 'direction': direction}
                            matched = True
                            
                            # Check if vehicle crosses start line
                            if obj['center'][1] < start_line['y'] and center[1] >= start_line['y']:
                                vehicle_frames[obj_id]['start'] = frame_count
                            
                            # Check if vehicle crosses end line
                            if obj['center'][1] < end_line['y'] and center[1] >= end_line['y']:
                                if 'start' in vehicle_frames[obj_id] and 'end' not in vehicle_frames[obj_id]:
                                    travel_frames = frame_count - vehicle_frames[obj_id]['start']
                                    travel_time = travel_frames / fps
                                    vehicle_frames[obj_id]['end'] = frame_count
                                    travel_times.append(travel_time)
                            
                            break
                    
                    if not matched:
                        new_id = max(tracked_objects.keys()) + 1 if tracked_objects else 0
                        current_objects[new_id] = {'box': current_box, 'center': center, 'counted': False, 'direction': direction}

                    # Display direction
                    cv2.putText(draw_frame, direction, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Count vehicles
        for obj_id, obj in current_objects.items():
            if not obj['counted'] and point_in_box(obj['center'], count_box):
                vehicle_count += 1
                obj['counted'] = True
        
        tracked_objects = current_objects

        # Calculate and display average time
        avg_time = sum(travel_times) / len(travel_times) if travel_times else 0
        cv2.putText(draw_frame, f"Avg Time: {avg_time:.2f}s", (width - 250, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display vehicle count
        cv2.putText(draw_frame, f"Vehicle Count: {vehicle_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Write frame to output video
        out.write(draw_frame)

        # Save frame as image every 30 frames
        if frame_count % 30 == 0:
            cv2.imwrite(f"saved_frames/frame_{frame_count}.jpg", draw_frame)

        print(f"Processed frame {frame_count}, Vehicle count: {vehicle_count}, Average time: {avg_time:.2f}s", end='\r')

    # Release resources
    cap.release()
    out.release()
    print(f"\nProcessing complete. Final vehicle count: {vehicle_count}, Final average time: {avg_time:.2f}s")

if __name__ == "__main__":
    main()


0: 384x640 (no detections), 68.0ms
Speed: 3.5ms preprocess, 68.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 1, Vehicle count: 0, Average time: 0.00s
0: 384x640 (no detections), 65.5ms
Speed: 2.4ms preprocess, 65.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 2, Vehicle count: 0, Average time: 0.00s
0: 384x640 1 car, 64.0ms
Speed: 1.9ms preprocess, 64.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 3, Vehicle count: 0, Average time: 0.00s
0: 384x640 1 car, 65.4ms
Speed: 2.9ms preprocess, 65.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 4, Vehicle count: 0, Average time: 0.00s
0: 384x640 2 cars, 63.2ms
Speed: 2.0ms preprocess, 63.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 5, Vehicle count: 0, Average time: 0.00s
0: 384x640 2 cars, 63.9ms
Speed: 2.5ms preprocess, 63.9ms inference, 1.0ms postproc